In [1]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
import sklearn.datasets as datasets
from sklearn.svm import SVC
import numpy as np

def main():
    mnist = datasets.fetch_mldata("MNIST original")
    X_all, y_all = mnist.data/255., mnist.target
    print("scaling")
    X = X_all[:60000, :]
    y = y_all[:60000]

    X_test = X_all[60000:, :]
    y_test = y_all[60000:]

    svm = SVC(cache_size=1000, kernel='rbf')

    parameters = {'C':10. ** np.arange(1,3), 'gamma':2. ** np.arange(-4, -1)}
    print("grid search")
    grid = GridSearchCV(svm, parameters, cv=StratifiedKFold(y, 3), verbose=3, n_jobs=-1)
    grid.fit(X, y)
    print("predicting")
    print("score: ", grid.score(X_test, y_test))
    print(grid.best_estimator_)

main()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


scaling
grid search
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] gamma=0.0625, C=10.0 ............................................
[CV] gamma=0.0625, C=10.0 ............................................
[CV] gamma=0.0625, C=10.0 ............................................
[CV] gamma=0.0625, C=10.0 ............................................


KeyboardInterrupt: 

In [ ]:
import struct

import numpy as np

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)